In [1]:
import numpy as np

# set the random seed
np.random.seed(42)

# import PennyLane
import pennylane as qml

In [2]:
from scipy.stats import unitary_group

# define the linear interferometer
U = unitary_group.rvs(4)
print(U)

[[ 0.23648826-0.48221431j  0.06829648+0.04447898j  0.51150074-0.09529866j
   0.55205719-0.35974699j]
 [-0.11148167+0.69780321j -0.24943828+0.08410701j  0.46705929-0.43192981j
   0.16220654-0.01817602j]
 [-0.22351926-0.25918352j  0.24364996-0.05375623j -0.09259829-0.53810588j
   0.27267708+0.66941977j]
 [ 0.11519953-0.28596729j -0.90164923-0.22099186j -0.09627758-0.13105595j
  -0.0200152 +0.12766128j]]


In [3]:
n_wires = 4
cutoff = 10

dev = qml.device("strawberryfields.gaussian", wires=n_wires, cutoff_dim=cutoff)


@qml.qnode(dev)
def gbs_circuit():
    # prepare the input squeezed states
    for i in range(n_wires):
        qml.Squeezing(1.0, 0.0, wires=i)

    # linear interferometer
    qml.InterferometerUnitary(U, wires=range(n_wires))
    return qml.probs(wires=range(n_wires))

In [16]:
probs = gbs_circuit()
print(probs.shape)

(10000,)


In [12]:
# Fock states to measure at output
measure_states = [(0, 0, 0, 0), (1, 1, 0, 0), (0, 1, 0, 1), (1, 1, 1, 1), (2, 0, 0, 0),(1, 0, 1, 0)]

# extract the probabilities of calculating several
# different Fock states at the output, and print them out
for i in measure_states:
    print(f"|{''.join(str(j) for j in i)}>: {probs[i]}")

IndexError: too many indices for array: array is 1-dimensional, but 4 were indexed

In [15]:
print(qml.draw(gbs_circuit)())

0: ──S(1.00,0.00)─╭U(M0)─┤ ╭Probs
1: ──S(1.00,0.00)─├U(M0)─┤ ├Probs
2: ──S(1.00,0.00)─├U(M0)─┤ ├Probs
3: ──S(1.00,0.00)─╰U(M0)─┤ ╰Probs


In [17]:
import pennylane as qml
import numpy as np

class UniversalQCSampler:
    """
    Sample taken from PennyLane's default.qubit simulator (using the Qubit-based approach)

    Parameters:
        nb_qubits (int, optional): the number of qubits of the circuit. Defaults to `5`.
        operation ("rotation" | "hadamard", optional): the gate applied on the qubits. Defaults to Hadamards.
        angle (float, optional): the rotation angle (around y-axis). Necessary in the case the operation is rotation.
    """

    def __init__(self, **kwargs):
        self.name = "UniversalQCSampler"
        self.nb_qubits = kwargs.get("nb_qubits", 5)
        self.operation = kwargs.get("operation", "hadamard")
        self.angle = kwargs.get("angle", None)

        if self.operation == "rotation" and self.angle is None:
            raise ValueError("You must define an angle when choosing the 'rotation' operation.")

        self.dev = qml.device("default.qubit", wires=self.nb_qubits, shots=1)

    def _simulate(self, length):
        shots = np.max([length // self.nb_qubits, 2])
        self.dev.shots = shots

        @qml.qnode(self.dev)
        def circuit():
            for i in range(self.nb_qubits):
                if self.operation == "hadamard":
                    qml.Hadamard(wires=i)
                elif self.operation == "rotation":
                    qml.RY(self.angle, wires=i)
                else:
                    raise NotImplementedError("Defined operation not implemented yet!")
            return qml.sample(qml.PauliZ(wires=range(self.nb_qubits)))

        return circuit()

    def sample(self, length):
        memory = self._simulate(length)
        bitstring_str = "".join(map(str, memory))
        return np.array(list(bitstring_str)).astype(np.int8)[:length]

# Example usage:
sampler = UniversalQCSampler(nb_qubits=3, operation="hadamard")
print(sampler.sample(10))


DeviceError: Shots must be a single non-negative integer or a sequence of non-negative integers.

In [62]:
import pennylane as qml
import numpy as np

class UniversalQCSampler:
    """
    Sample taken from PennyLane's default.qubit simulator (using the Qubit-based approach)
    """

    def __init__(self, **kwargs):
        self.name = "UniversalQCSampler"
        self.nb_qubits = kwargs.get("nb_qubits", 5)
        self.operation = kwargs.get("operation", "hadamard")
        self.angle = kwargs.get("angle", None)

        if self.operation == "rotation" and self.angle is None:
            raise ValueError("You must define an angle when choosing the 'rotation' operation.")

        # Set up the device with a default number of shots
        self.default_shots = 1000
        self.dev = qml.device("default.qubit", wires=self.nb_qubits, shots=self.default_shots)

    def simulate(self, length):
        shots = np.max([length // self.nb_qubits, 2])
    
        @qml.qnode(self.dev)
        def circuit():
            for i in range(self.nb_qubits):
                if self.operation == "hadamard":
                    qml.Hadamard(wires=i)
                elif self.operation == "rotation":
                    qml.RY(self.angle, wires=i)
                else:
                    raise NotImplementedError("Defined operation not implemented yet!")
            #print(qml.draw(circuit)())
            return [qml.sample(qml.PauliZ(wires=i)) for i in range(self.nb_qubits)]
        
        #(qml.draw(circuit)())
        # Run the circuit multiple times to get the desired number of samples
        result = []
        for _ in range(shots):
            result.extend(circuit())
    
        return np.array(result)


    def sample(self, length):
        memory = self.simulate(length).flatten()
        # Map -1 to 0 and +1 to 1
        memory = (memory + 1) // 2
        return memory[:length]



# Example usage:
sampler = UniversalQCSampler(nb_qubits=3, operation="rotation", angle = np.pi/2)
print(sampler.sample(10))


0: ──RY(1.57)─┤  Sample[Z]
1: ──RY(1.57)─┤  Sample[Z]
2: ──RY(1.57)─┤  Sample[Z]
[1 0 1 0 1 0 1 1 0 0]


In [ ]:
sampler.simulate()